## 1. Data Preparation

### 1.1. Import Libraries

In [24]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

- Import data, check for comlumns and variables
### 1.2. Load and Clean Data
Load the dataset, inspect its structure, and perform initial cleaning. We drop the original 'Price' column (in Toman) and use 'Price(USD)' for our analysis, renaming it to 'Price'.

In [25]:
houses = pd.read_csv("./data/housePrice.csv")
houses = houses.drop('Price', axis=1)
houses = houses.rename(columns={'Price(USD)': 'Price'})
houses.info()
houses.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3479 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3479 non-null   object 
 1   Room       3479 non-null   int64  
 2   Parking    3479 non-null   bool   
 3   Warehouse  3479 non-null   bool   
 4   Elevator   3479 non-null   bool   
 5   Address    3456 non-null   object 
 6   Price      3479 non-null   float64
dtypes: bool(3), float64(1), int64(1), object(2)
memory usage: 119.0+ KB


,Area,Room,Parking,Warehouse,Elevator,Address,Price
0,63,1,True,True,True,Shahran,61666.67
1,60,1,True,True,True,Shahran,61666.67
2,79,2,True,True,True,Pardis,18333.33
3,95,2,True,True,True,Shahrake Qods,30083.33
4,123,2,True,True,True,Shahrake Gharb,233333.33


- Since the Address column has some missing data in certain rows, we will drop these rows.

In [26]:
houses = houses.dropna(subset=['Address'])

- Check the header again

In [27]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3456 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3456 non-null   object 
 1   Room       3456 non-null   int64  
 2   Parking    3456 non-null   bool   
 3   Warehouse  3456 non-null   bool   
 4   Elevator   3456 non-null   bool   
 5   Address    3456 non-null   object 
 6   Price      3456 non-null   float64
dtypes: bool(3), float64(1), int64(1), object(2)
memory usage: 145.1+ KB


- That said, the data is now complete. HOWEVER, we have not yet categorized the data into the two main types: Numerical and Categorical. Now, we will do that.

In [28]:
# Identify numerical and categorical features
numerical_features = ['Price', 'Area']
categorical_features = ['Address', 'Parking', 'Warehouse', 'Elevator', 'Room']

# Convert 'Area' to numeric if possible (it's showing as 'object' type)
houses['Area'] = pd.to_numeric(houses['Area'], errors='coerce')
if houses['Area'].isna().sum() == 0:  # If conversion worked with no NaN values
    numerical_features.append('Area')
else:
    categorical_features.append('Area')
    
# Convert boolean columns to categorical type
for col in ['Parking', 'Warehouse', 'Elevator']:
    houses[col] = houses[col].astype('category')

# Convert Room to categorical if needed for certain analyses
houses['Room'] = houses['Room'].astype('int64')  # Ensure it's integer type

# Display the data types after conversion
print("Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)
print("\nDataFrame Info after type conversion:")
houses.info()

Numerical Features: ['Price', 'Area']
Categorical Features: ['Address', 'Parking', 'Warehouse', 'Elevator', 'Room', 'Area']

DataFrame Info after type conversion:
<class 'pandas.core.frame.DataFrame'>
Index: 3456 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Area       3450 non-null   float64 
 1   Room       3456 non-null   int64   
 2   Parking    3456 non-null   category
 3   Warehouse  3456 non-null   category
 4   Elevator   3456 non-null   category
 5   Address    3456 non-null   object  
 6   Price      3456 non-null   float64 
dtypes: category(3), float64(2), int64(1), object(1)
memory usage: 145.4+ KB


### 1.3. Outlier Removal
To prevent extreme values from skewing the analysis, we remove the top and bottom 1% of properties based on price.

In [29]:
# Calculate 1st and 99th percentiles
lower_percentile = houses['Price'].quantile(0.01)
upper_percentile = houses['Price'].quantile(0.99)

print(f"1st percentile (1% lowest): ${lower_percentile:.2f}")
print(f"99th percentile (1% highest): ${upper_percentile:.2f}")

# Remove outliers
houses_before = len(houses)
houses = houses[(houses['Price'] >= lower_percentile) & (houses['Price'] <= upper_percentile)]
houses_after = len(houses)

print(f"\nDataset size before removing outliers: {houses_before}")
print(f"Dataset size after removing outliers: {houses_after}")
print(f"Removed {houses_before - houses_after} outliers ({((houses_before - houses_after) / houses_before * 100):.1f}%)")

1st percentile (1% lowest): $9833.33
99th percentile (1% highest): $1333333.33

Dataset size before removing outliers: 3456
Dataset size after removing outliers: 3395
Removed 61 outliers (1.8%)


In [30]:
avg_price_all_locations = houses.groupby('Address')['Price'].mean().sort_values(ascending=False)

for i, (location, price) in enumerate(avg_price_all_locations.items(), 1):
    property_count = houses[houses['Address'] == location].shape[0]
    print(f"{i:3d}. {location:<25} ${price:>10,.0f} ({property_count:>2} properties)")


  1. Lavasan                   $   866,667 ( 2 properties)
  2. Argentina                 $   838,833 ( 2 properties)
  3. Elahieh                   $   706,829 (14 properties)
  4. Farmanieh                 $   645,423 (54 properties)
  5. Kamranieh                 $   610,048 (14 properties)
  6. Zaferanieh                $   589,032 (23 properties)
  7. Ajudaniye                 $   555,333 ( 5 properties)
  8. Niavaran                  $   553,327 (63 properties)
  9. Velenjak                  $   530,833 (18 properties)
 10. Aqdasieh                  $   507,083 (24 properties)
 11. Mahmoudieh                $   423,333 ( 2 properties)
 12. Araj                      $   404,444 ( 9 properties)
 13. Shahrake Gharb            $   398,996 (59 properties)
 14. Dezashib                  $   395,333 ( 9 properties)
 15. Gheitarieh                $   357,012 (141 properties)
 16. Dorous                    $   353,333 (11 properties)
 17. Saadat Abad               $   316,292 (129 propert